# 1. Récupération des données


## a) Utilisation de l'API Yelp

> On envoie de requêtes **GET** pour obtenir les commentaires et les photos de 200 restaurants de Paris.

In [1]:
import os 
import requests
import uuid
import json
import pandas as pd


key_var_name      = 'YELP_KEY'
endpoint_var_name = 'YELP_ENDPOINT'

if not key_var_name in os.environ:
    raise Exception('Please set/export the environment variable 1: {}'\
                    .format(key_var_name))
yelp_key = os.environ[key_var_name]

if not endpoint_var_name in os.environ:
    raise Exception('Please set/export the environment variable 2: {}'\
                    .format(endpoint_var_name))
endpoint = os.environ[endpoint_var_name]


path = '/businesses/search'
constructed_url = endpoint + path

headers = {'Authorization': 'Bearer %s' % yelp_key}

params = {'term':'restaurants',
          'location':'Paris',
          'offset':0, 'limit':50}

request = requests.get(constructed_url, params=params,
                       headers=headers)

print('The status code is {}'.format(request.status_code))

reponse = request.text
businesses = json.loads(reponse)['businesses']

The status code is 200


## b) Enregistrement des données dans un dictionnaire

> On ne récupère que les mauvais avis d'environ 200 restaurants de Paris. **Le rating** des commentaires est donc strictement inférieur à 3.

In [3]:
data = {'id': [], 'name': [], 'text_review': [], 'photo_1': [],
        'photo_2': [], 'photo_3': []}

data_1 = {'id': [], 'name': [], 'text_review': [], 'rating': []}


for elt in businesses:
    url = 'https://api.yelp.com/v3/businesses/' + elt['id'] + '/reviews'
    params = {'locale':'fr_FR'}
    req = requests.get(url, headers=headers, params=params)
    if req.status_code == 200:
        reponse = req.text
        reviews = json.loads(reponse)['reviews']
        text_reviews   = ""
        text_reviews_l = ""    
        for elt_r in reviews:
            text_reviews = text_reviews + " " + elt_r['text']
        
        text_reviews_l = text_reviews
        url_1 = 'https://api.yelp.com/v3/businesses/' + elt['id']
        req_1 = requests.get(url_1, headers=headers)
        if req_1.status_code == 200:
            reponse_1 = req_1.text
            names = json.loads(reponse_1)['name']
            photos = json.loads(reponse_1)['photos']
            if (len(photos) == 3 and len(text_reviews_l) != 0):
                data['id'].append(elt['id'])
                data['name'].append(names)
                data['text_review'].append(text_reviews_l)
                data['photo_1'].append(photos[0])
                data['photo_2'].append(photos[1])
                data['photo_3'].append(photos[2])
                for elt_r in reviews:
                    data_1['id'].append(elt['id'])
                    data_1['name'].append(names)
                    data_1['text_review'].append(elt_r['text'])
                    data_1['rating'].append(elt_r['rating'])

* Récupération de 200 restaurants avec des avis négatifs :

In [4]:
i = 0
while len(data['id']) <= 200:
    print(len(data['id']))    
    i += 50
    params = {'term':'restaurants', 'location':'Paris', 
                  'offset':i, 'limit':50}

    request = requests.get(constructed_url, params=params,
                               headers=headers)
    if request.status_code == 200:
        
        reponse = request.text
        businesses = json.loads(reponse)['businesses']


        for elt in businesses:
            url = 'https://api.yelp.com/v3/businesses/' + elt['id'] + '/reviews'
            params = {'locale':'fr_FR'}
            req = requests.get(url, headers=headers, params=params)
            if req.status_code == 200:
                reponse = req.text
                reviews = json.loads(reponse)['reviews']
                text_reviews   = ""
                text_reviews_l = ""    
                for elt_r in reviews:
                    text_reviews = text_reviews + " " + elt_r['text']
        
                text_reviews_l = text_reviews
                url_1 = 'https://api.yelp.com/v3/businesses/' + elt['id']
                req_1 = requests.get(url_1, headers=headers)
                if req_1.status_code == 200:
                    reponse_1 = req_1.text
                    names = json.loads(reponse_1)['name']
                    photos = json.loads(reponse_1)['photos']
                    if (len(photos) == 3 and len(text_reviews_l) != 0):
                        data['id'].append(elt['id'])
                        data['name'].append(names)
                        data['text_review'].append(text_reviews_l)
                        data['photo_1'].append(photos[0])
                        data['photo_2'].append(photos[1])
                        data['photo_3'].append(photos[2])
                        for elt_r in reviews:
                            data_1['id'].append(elt['id'])
                            data_1['name'].append(names)
                            data_1['text_review'].append(elt_r['text'])
                            data_1['rating'].append(elt_r['rating'])

48
91
138
183


## c) Conversion en fichier csv

In [5]:
data_df = pd.DataFrame.from_dict(data)
data_df1 = pd.DataFrame.from_dict(data_1)

print(data_df.shape)
data_df.head()

(230, 6)


,id,name,text_review,photo_1,photo_2,photo_3
0,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Une adresse immanquable. \nLe meilleur magret...,https://s3-media2.fl.yelpcdn.com/bphoto/Je6THJ...,https://s3-media2.fl.yelpcdn.com/bphoto/Y0D70M...,https://s3-media4.fl.yelpcdn.com/bphoto/HetBW7...
1,IU9_wVOGBKjfqTTpAXpKcQ,Bistro des Augustins,Voilà un bar qu on aime voir à Paris !\nL acc...,https://s3-media1.fl.yelpcdn.com/bphoto/hPCZTb...,https://s3-media4.fl.yelpcdn.com/bphoto/Y5fZV7...,https://s3-media2.fl.yelpcdn.com/bphoto/mYdGVn...
2,FFz-WusZrBYZexKqhqzCkg,L'As du Fallafel,"Rien a ajouter, ils sont toujours parfaits le...",https://s3-media3.fl.yelpcdn.com/bphoto/QMNELS...,https://s3-media1.fl.yelpcdn.com/bphoto/xtLO-X...,https://s3-media4.fl.yelpcdn.com/bphoto/xIxo5a...
3,cEjF41ZQB8-SST8cd3EsEw,L'Avant Comptoir,"Nous sommes complètement déstabilisé, possibl...",https://s3-media3.fl.yelpcdn.com/bphoto/mVwgxg...,https://s3-media3.fl.yelpcdn.com/bphoto/XA5QGI...,https://s3-media2.fl.yelpcdn.com/bphoto/azxglN...
4,26Ts-2XRSeqiweSwLuyv6Q,Grenouilles,Nous quittons à regret cette très sympathique...,https://s3-media4.fl.yelpcdn.com/bphoto/fgVrLC...,https://s3-media3.fl.yelpcdn.com/bphoto/dyUbhh...,https://s3-media1.fl.yelpcdn.com/bphoto/pJOb8D...


In [6]:
print(data_df1.shape)
data_df1.head()

(644, 4)


,id,name,text_review,rating
0,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Une adresse immanquable. \nLe meilleur magret ...,5
1,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,"Très bon, le prix abordable et le restaurant e...",4
2,-0iLH7iQNYtoURciDpJf6w,Le Comptoir de la Gastronomie,Très bon accueil \nTrès bonne cuisine,5
3,IU9_wVOGBKjfqTTpAXpKcQ,Bistro des Augustins,Voilà un bar qu on aime voir à Paris !\nL accu...,5
4,IU9_wVOGBKjfqTTpAXpKcQ,Bistro des Augustins,Super resto juste au bord de la Seine avec Not...,5


In [7]:
data_df.to_csv("P6_01_fichier.csv", sep=',', index=False)
data_df1.to_csv("P6_01_fichier_rating.csv", sep=',', index=False)